In [3]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.framework import ops

In [4]:
dataset = pd.read_csv("dataset.csv")

In [5]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61.0,278.0,1.0
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101.0,1028.0,1.0
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485.0,2259.0,1.0
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40.0,191.0,1.0
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40.0,191.0,1.0


In [6]:
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1:].values

In [7]:
print(X.shape, Y.shape)

(4601, 57) (4601, 1)


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [9]:
X_train = X_train.T
Y_train = Y_train.T
X_test = X_test.T
Y_test = Y_test.T
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

X_train shape:  (57, 3450)
Y_train shape:  (1, 3450)
X_test shape:  (57, 1151)
Y_test shape:  (1, 1151)


In [10]:
def create_placeholders(n_x, n_y):
    """
    n_x -- scalar, number of features in X
    n_y -- scalar, number of classes in output
    
    """
    X = tf.placeholder(tf.float32, shape=(n_x, None), name='X')
    Y = tf.placeholder(tf.float32, shape=(n_y, None), name='Y')
    return X, Y

In [11]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [12]:
def initialize_parameters(layer_dims):
    L=len(layer_dims) -1
    parameters={}
    for l in range(1, L+1):
        parameters['W'+str(l)]=tf.get_variable('W'+str(l), [layer_dims[l], layer_dims[l-1]], initializer=tf.contrib.layers.xavier_initializer())
        parameters['b'+str(l)]=tf.get_variable('b'+str(l), [layer_dims[l], 1], initializer=tf.zeros_initializer())
    return parameters

In [13]:
tf.reset_default_graph()
with tf.Session() as sess:
    parameters=initialize_parameters([57, 25, 12, 1])
    for l in range(1, 4):
        print('W'+str(l)+' = '+str(parameters['W'+str(l)]))
        print('b'+str(l)+' = '+str(parameters['b'+str(l)]))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1 = <tf.Variable 'W1:0' shape=(25, 57) dtype=float32_ref>
b1 = <tf.Variable 'b1:0' shape=(25, 1) dtype=float32_ref>
W2 = <tf.Variable 'W2:0' shape=(12, 25) dtype=float32_ref>
b2 = <tf.Variable 'b2:0' shape=(12, 1) dtype=float32_ref>
W3 = <tf.Variable 'W3:0' shape=(1, 12) dtype=float32_ref>
b3 = <tf.Variable 'b3:0' shape=(1, 1) dtype=float32_ref>


In [14]:
def forward_propagation(X, parameters):
    """
    returns -- ZL, output of the last linear unit
    
    """
    L = len(parameters)//2
    A = X
    for l in range(1, L):
        W = parameters['W'+str(l)]
        b = parameters['b'+str(l)]
        Z = tf.matmul(W, A) + b
        A = tf.nn.relu(Z)
    Z = tf.matmul(parameters['W'+str(L)], A) + parameters['b'+str(L)]
    return Z

In [15]:
tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholders(57, 1)
    parameters = initialize_parameters([57, 25, 12, 1])
    Z3 = forward_propagation(X, parameters)
    print("Z3 = " + str(Z3))

Z3 = Tensor("add_2:0", shape=(1, ?), dtype=float32)


In [16]:
def compute_cost(ZL, Y):
    """
    Computes the cost
    
    Arguments:
    ZL -- output of forward propagation (output of the last LINEAR unit), of shape (num_classes, number of examples)
    Y -- "true" labels vector placeholder, same shape as ZL
    First take transpose to use tensorflow functions
    Returns:
    cost - Tensor of the cost function
    """
    logits = tf.transpose(ZL)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
    return cost

In [17]:
tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholders(57, 1)
    parameters = initialize_parameters([57, 25, 12, 1])
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    print("cost = " + str(cost))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
cost = Tensor("Mean:0", shape=(), dtype=float32)


In [18]:
def model(X_train, Y_train, X_test, Y_test, layers_dims, learning_rate=0.0001, num_epochs=1500, minibatch_size=32, print_cost=True):
    ops.reset_default_graph()
    (n_x, m) = X_train.shape
    n_y = Y_train.shape[0]
    costs=[]
    X, Y = create_placeholders(n_x, n_y)
    parameters = initialize_parameters(layers_dims)
    ZL = forward_propagation(X, parameters)
    cost = compute_cost(ZL, Y)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            epoch_cost = 0
            num_minibatches = m//minibatch_size
            minibatches = random_mini_batches(X_train , Y_train, minibatch_size)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _, minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                epoch_cost += minibatch_cost/num_minibatches
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")
        correct_prediction = tf.equal(tf.to_int32(ZL>0.5), tf.to_int32(Y))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        return parameters

In [ ]:
parameters = model(X_train, Y_train, X_test, Y_test, [57, 28, 16, 8, 1], 0.001, 500, 32)

In [20]:
def forward_propagation_for_predict(X, parameters):
    """
    returns -- ZL, output of the last linear unit
    
    """
    L = len(parameters)//2
    A = X
    for l in range(1, L):
        W = parameters['W'+str(l)]
        b = parameters['b'+str(l)]
        Z = tf.matmul(tf.cast(W, tf.float32), tf.cast(A, tf.float32)) + tf.cast(b, tf.float32)
        A = tf.nn.relu(Z)
    Z = tf.matmul(parameters['W'+str(L)], A) + parameters['b'+str(L)]
    return Z

In [21]:
def predict(X, parameters):
    L = len(parameters)//2
    params = {}
    for l in range(1, L+1):
        params['W'+str(l)] = tf.convert_to_tensor(parameters['W'+str(l)])
        params['b'+str(l)] = tf.convert_to_tensor(parameters['b'+str(l)])
    
    m = X.shape[1]
    x = tf.placeholder("float32", [57, m])
    
    zL = forward_propagation_for_predict(X, params)
    p = tf.to_int32(zL>0.5)
    
    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x: X})
        
    return prediction.T.reshape(1, m)

In [22]:
pred=predict(X_test, parameters)
print(pred.shape)

(1, 1151)


In [23]:
idx=505
print('pred =', pred[0, idx])
print ("y = " + str(np.squeeze(Y_test[:, idx])))

pred = 1
y = 1.0


In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [25]:
Y_t = Y_test.T
pred_t = pred.T
results = confusion_matrix(Y_t, pred_t)
acc_score = accuracy_score(Y_t, pred_t)
class_report = classification_report(Y_t, pred_t)
print("Confusion Matrix:\n", results)
print("Accuracy: ", acc_score)

Confusion Matrix:
 [[666  25]
 [ 57 403]]
Accuracy:  0.9287576020851434
